---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
print('X_train first entry: \n\n', X_train.iloc[0])
print('\n\n X_train shape: ', X_train.shape)

X_train first entry: 

 I'll text you when I drop x off


 X_train shape:  (4179,)


### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [5]:
def answer_one():
    a = spam_data.groupby(spam_data['target'] == 1).count()
    return 100*a['target'][1]/(a['target'][0]+a['target'][1]) #Your answer here

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)

    return max(vect.get_feature_names(), key = len)#Your answer here

In [20]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    #transform data from text into numeric representations
    vect = CountVectorizer().fit(X_train)
    
    #transform the training data into a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    
    #called multinomial naive bayes classifier
    model = MultinomialNB(alpha = 0.1)
    
    #train model using multinomiaNB() model with the document-term matrix
    model.fit(X_train_vectorized, y_train)
    
    #predict the transformed test documents
    prediction = model.predict(vect.transform(X_test))
    
    #auc curve
    a = roc_auc_score(y_test, prediction)
    return a #Your answer here

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vect.get_feature_names())
    X_train_vectorized = vect.transform(X_train)
    
    # sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    # Get largest tfidf values across all documents.
    max_tfidf = X_train_vectorized.max(0).toarray()[0]
    #sorted index
    sorted_tfidf_index = max_tfidf.argsort()
    #sorted values
    sorted_tfidf_value = max_tfidf[sorted_tfidf_index]
    
    smallest_tfidf = pd.Series(sorted_tfidf_value[:20], index = feature_names[sorted_tfidf_index[:20]])
    largest_tfidf = pd.Series(sorted_tfidf_value[-20:][::-1], index = feature_names[sorted_tfidf_index[-20:][::-1]])
    return (smallest_tfidf, largest_tfidf) #Your answer here

In [10]:
answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [93]:
def answer_five():
    #fit and transform the training data into a document-term matrix
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    #called multinomial naive bayes classifier
    model = MultinomialNB(alpha = 0.1)
    
    #train model using multinomiaNB() model with the document-term matrix
    model.fit(X_train_vectorized, y_train)
    
    #predict the transformed test documents
    prediction = model.predict(vect.transform(X_test))
    return roc_auc_score(y_test, prediction)#Your answer here

In [94]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [10]:
def answer_six():
    char_not_spam = 0
    char_spam = 0
    
    spam = spam_data[spam_data['target'] == 1]
    spam = spam.reset_index()
    for x in range(0,len(spam)):
        char_spam += len(spam['text'][x])
        
    not_spam = spam_data[spam_data['target'] == 0]
    not_spam = not_spam.reset_index()
    for x in range(0,len(not_spam)):
        char_not_spam += len(not_spam['text'][x])
    
    avg_len_spam = char_spam/len(spam)
    avg_len_not_spam = char_not_spam/len(not_spam)
    return (avg_len_not_spam, avg_len_spam)#Your answer here

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [11]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
from sklearn.svm import SVC
def answer_seven():
    # fit and transform the training data into a document-term matrix
    vect = TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #the length of document
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    #add features
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    
    # svc model
    model = SVC(C=10000)
    model.fit(X_train_vectorized, y_train)
    prediction = model.predict(X_test_vectorized)
    
    return roc_auc_score(y_test, prediction)#Your answer here

In [35]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [13]:
import re
def answer_eight():
    spam = spam_data[spam_data['target'] == 1]
    not_spam = spam_data[spam_data['target'] == 0]
    
    #count digits in text
    digits_spam = [sum(char.isnumeric() for char in x) for x in spam['text']]
    digits_not_spam = [sum(char.isnumeric() for char in x) for x in not_spam['text']]
    
    #avg number of digits
    avg_digits_spam = np.mean(digits_spam)
    avg_digits_not_spam = np.mean(digits_not_spam)
    
    return (avg_digits_not_spam, avg_digits_spam) #Your answer here


In [96]:
answer_eight()

(0.2992746113989637, 15.76037483266399)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [81]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    # fit and transform the training data into a document-term matrix
    vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #count digits in text
    digits_train = [X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))]
    digits_test = [X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()])))]

    #the length of document
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    #add features
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_train_vectorized = add_feature(X_train_vectorized, digits_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    X_test_vectorized = add_feature(X_test_vectorized, digits_test)
    # logistic model
    model = LogisticRegression(C=100)
    model.fit(X_train_vectorized, y_train)
    prediction = model.predict(X_test_vectorized)
    return roc_auc_score(y_test, prediction)#Your answer here

In [82]:
answer_nine()

/Users/jinlingxing/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9809793219360643

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [70]:
def answer_ten():
    spam_data['non word length'] = spam_data['text'].str.findall(r'\W').str.len()
    avg_non_word_spam = np.mean(spam_data['non word length'][spam_data['target']==1])
    avg_non_word_not_spam = np.mean(spam_data['non word length'][spam_data['target']==0])
    return (avg_non_word_not_spam, avg_non_word_spam)#Your answer here

In [71]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [92]:
def answer_eleven():
    vect = CountVectorizer(min_df = 5, ngram_range = (2,5), analyzer = 'char_wb')
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.fit_transform(X_test)

    #add features
    X_train_vectorized_add_features = add_feature(X_train_vectorized, [X_train.str.len(),
                                                                      X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                                      X_train.str.findall(r'\W').str.len()])
    
    X_test_vectorized_add_features = add_feature(X_test_vectorized, [X_test.str.len(),
                                                       X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                       X_test.str.findall(r'\W').str.len()])
    
    #train model
    model = LogisticRegression(C=100)
    model.fit(X_train_vectorized_add_features, y_train)
    prediction = model.predict(X_test_vectorized_add_features)
    auc = roc_auc_score(y_test, prediction)
    
    #find the 10 smallest and 10 largest coefficients from the model
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]]
    largest = feature_names[sorted_coef_index[:-11:-1]]
    return (auc, list(smallest), list(largest))#Your answer here

In [91]:
answer_eleven()

/Users/jinlingxing/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ValueError: X has 7677 features per sample; expecting 16317